In [1]:
import pandas as pd
import pymysql
import os

# List of CSV files and their corresponding MySQL table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('order_items.csv', 'order_items'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments')
]

# Connect to the MySQL database using pymysql
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='pankaj1234',
    database='ecom'
)
cursor = conn.cursor()

# Path to folder containing the CSV files
folder_path = 'C:/Users/j s/Desktop/JN/E. Com'

# Helper function to map pandas dtypes to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

# Process each CSV and insert into MySQL
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read CSV into DataFrame
    df = pd.read_csv(file_path)

    # Clean column names
    df.columns = [col.strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Replace NaN with None for SQL NULL compatibility
    df = df.where(pd.notnull(df), None)

    # Generate and execute CREATE TABLE query
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_query)

    # Insert records row-by-row
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        placeholders = ', '.join(['%s'] * len(values))
        columns_formatted = ', '.join([f'`{col}`' for col in df.columns])
        insert_query = f'INSERT INTO `{table_name}` ({columns_formatted}) VALUES ({placeholders})'
        cursor.execute(insert_query, values)
    
    # Commit after each file
    conn.commit()
    print(f"[✓] Imported: {csv_file} into `{table_name}`")

# Close connection
cursor.close()
conn.close()
print("\nAll files imported successfully.")


[✓] Imported: customers.csv into `customers`
[✓] Imported: orders.csv into `orders`
[✓] Imported: order_items.csv into `order_items`
[✓] Imported: sellers.csv into `sellers`
[✓] Imported: products.csv into `products`
[✓] Imported: geolocation.csv into `geolocation`
[✓] Imported: payments.csv into `payments`

All files imported successfully.
